In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet 


In [74]:


with open("popular_anime.csv", "rb") as f:
    result = chardet.detect(f.read())


print(result)

df=pd.read_csv("popular_anime.csv",encoding="utf-8")
print(df.columns)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
Index(['id', 'name', 'genres', 'type', 'episodes', 'status', 'aired_from',
       'aired_to', 'duration_per_ep', 'score', 'scored_by', 'rank', 'rating',
       'studios', 'producers', 'image', 'trailer', 'synopsis'],
      dtype='object')


In [75]:
emptydatapercentage=df.isnull().mean()*100
print(emptydatapercentage.sort_values(ascending=False))


trailer            81.009540
aired_to           62.081526
producers          53.332177
studios            41.245447
score              35.476149
scored_by          35.476149
rank               23.809193
genres             21.054640
synopsis           18.622723
aired_from          3.313096
episodes            2.702515
rating              2.511709
type                0.301821
name                0.000000
duration_per_ep     0.000000
status              0.000000
image               0.000000
id                  0.000000
dtype: float64


In [76]:
cols_to_drop=emptydatapercentage[emptydatapercentage>60].index
print(cols_to_drop)



Index(['aired_to', 'trailer'], dtype='object')


In [77]:
print(type(cols_to_drop))
print(cols_to_drop)


<class 'pandas.core.indexes.base.Index'>
Index(['aired_to', 'trailer'], dtype='object')


In [78]:
print(df.columns.tolist())


['id', 'name', 'genres', 'type', 'episodes', 'status', 'aired_from', 'aired_to', 'duration_per_ep', 'score', 'scored_by', 'rank', 'rating', 'studios', 'producers', 'image', 'trailer', 'synopsis']


In [79]:
print(df.dtypes)

id                   int64
name                object
genres              object
type                object
episodes           float64
status              object
aired_from          object
aired_to            object
duration_per_ep     object
score              float64
scored_by          float64
rank               float64
rating              object
studios             object
producers           object
image               object
trailer             object
synopsis            object
dtype: object


In [80]:
df['aired_from'] = pd.to_datetime(df['aired_from'], errors='coerce')


# duration_per_ep'deki süreyi dakikaya çevir
df['duration_per_ep_minutes'] = df['duration_per_ep'].str.extract('(\\d+)').astype(float)

# Kategorik sütunları dönüştür
df['status'] = df['status'].astype('category')
df['type'] = df['type'].astype('category')
df['rating'] = df['rating'].astype('category')
df['genres'] = df['genres'].astype('category')




In [81]:
print(df.dtypes)

id                                       int64
name                                    object
genres                                category
type                                  category
episodes                               float64
status                                category
aired_from                 datetime64[ns, UTC]
aired_to                                object
duration_per_ep                         object
score                                  float64
scored_by                              float64
rank                                   float64
rating                                category
studios                                 object
producers                               object
image                                   object
trailer                                 object
synopsis                                object
duration_per_ep_minutes                float64
dtype: object


In [82]:
print(emptydatapercentage.sort_values(ascending=False))

trailer            81.009540
aired_to           62.081526
producers          53.332177
studios            41.245447
score              35.476149
scored_by          35.476149
rank               23.809193
genres             21.054640
synopsis           18.622723
aired_from          3.313096
episodes            2.702515
rating              2.511709
type                0.301821
name                0.000000
duration_per_ep     0.000000
status              0.000000
image               0.000000
id                  0.000000
dtype: float64


In [83]:
df.drop(columns=['studios','producers','aired_to','image','trailer'],inplace=True)

In [84]:
for col in ['genres', 'type', 'status', 'rating']:
    if col in df.columns and df[col].dtype.name == 'category':
        if 'Unknown' not in df[col].cat.categories:
            df[col] = df[col].cat.add_categories(['Unknown'])
    df[col] = df[col].fillna('Unknown')
for col in ['episodes', 'duration_per_ep_minutes','score','scored_by','rank','rating']:
    if col in df.columns and df[col].dtype.name == 'category':
        if '-1,0,23' not in df[col].cat.categories:
            df[col] = df[col].cat.add_categories([-1,0,23])
    

df.fillna({
    'name': 'Unknown',
    'genres': 'Unknown',
    'type': df['type'].mode()[0],
    'episodes': 23,
    'status': df['status'].mode()[0],
    'aired_from': pd.Timestamp('2000-01-01'),
    'aired_to': pd.Timestamp('2000-01-01'),
    'duration_per_ep_minutes': 24,
    'score': 0,
    'scored_by': df['scored_by'].median(),
    'rank': -1,
    'rating': 0,
    'image': 'No image',
    'trailer': 'No trailer',
    'synopsis': 'No synopsis available'
}, inplace=True)


In [85]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28825 entries, 0 to 28824
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   id                       28825 non-null  int64   
 1   name                     28825 non-null  object  
 2   genres                   28825 non-null  category
 3   type                     28825 non-null  category
 4   episodes                 28825 non-null  float64 
 5   status                   28825 non-null  category
 6   aired_from               28825 non-null  object  
 7   duration_per_ep          28825 non-null  object  
 8   score                    28825 non-null  float64 
 9   scored_by                28825 non-null  float64 
 10  rank                     28825 non-null  float64 
 11  rating                   28825 non-null  category
 12  synopsis                 28825 non-null  object  
 13  duration_per_ep_minutes  28825 non-null  float64 
dtypes: cat

In [86]:
print(df['type'].value_counts())


type
TV            8249
Movie         4836
OVA           4166
Music         4087
ONA           4076
Special       1779
TV Special     745
CM             504
PV             296
Unknown         87
Name: count, dtype: int64


In [87]:
for col in df.select_dtypes(include=['object', 'category']):
    print(f"--- {col} ---")
    print(df[col].value_counts())
    print("\n")


--- name ---
name
Spirit Guardians                               5
Promise                                        5
Happy Heroes: Rebel Rescue                     4
Cyborg 009                                     4
Meow Meow Japanese History                     4
                                              ..
Pyuu to Fuku! Jaguar: Ima, Fuki ni Yukimasu    1
Argento Soma: Hitori to Hitori                 1
Yurumates                                      1
The Fleet of the Rising Sun                    1
Marin X                                        1
Name: count, Length: 28350, dtype: int64


--- genres ---
genres
Unknown                                            6069
Comedy                                             2621
Fantasy                                            1462
Hentai                                             1293
Slice of Life                                       821
                                                   ... 
Boys Love, Drama, Horror, Mystery, Superna

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

In [89]:
def clean_anime_name(name):
    name = re.sub(r"Season\s*\d+", "", name, flags=re.IGNORECASE)
    name = re.sub(r"Final\s*Season.*", "", name, flags=re.IGNORECASE)
    name = re.sub(r"Part\s*\d+", "", name, flags=re.IGNORECASE)
    name = re.sub(r"\s*:\s*.*$", "", name, flags=re.IGNORECASE)  # isimden sonrasını sil
    name = name.strip()
    return name


In [90]:
# Dataset yükle
df = pd.read_csv('popular_anime.csv')

# synopsis sütununu doldur
df['synopsis'] = df['synopsis'].fillna('')


In [91]:
df['name_clean'] = df['name'].apply(clean_anime_name)
df['name_lower'] = df['name'].str.lower()


In [92]:
def map_type_group(x):
    if x in ['TV', 'OVA', 'ONA', 'Special']:
        return 'TV_GROUP'
    elif x == 'Movie':
        return 'MOVIE_GROUP'
    else:
        return 'OTHER'

df['type_group'] = df['type'].apply(map_type_group)


In [93]:
# NaN isimleri sil
df_clean = df.dropna(subset=['name_clean'])

# en yüksek skoru al
df_dedup = (
    df_clean
    .sort_values('score', ascending=False)
    .drop_duplicates(subset=['name_clean', 'type_group'], keep='first')
)

# TV ve Movie DataFrame'leri oluştur
tv_final_df = df_dedup[df_dedup['type_group'] == 'TV_GROUP']
movie_final_df = df_dedup[df_dedup['type_group'] == 'MOVIE_GROUP']


In [94]:
# TV synopsis listesi
tv_synopsis_list = tv_final_df['synopsis'].tolist()
# 1. Veri temizliği (zaten yapmışsın ama synopsis uzunluk filtresi ekleniyor)

df_filtered = df[df['synopsis'].str.len() >= 30].copy()
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df_filtered['synopsis'])

# TF-IDF vektörizer
vectorizer = TfidfVectorizer(stop_words='english')

# Movie synopsis listesi
movie_synopsis_list = movie_final_df['synopsis'].tolist()

vectorizer_movie = TfidfVectorizer(stop_words='english')
tfidf_matrix_movie = vectorizer_movie.fit_transform(movie_synopsis_list)


In [95]:
!pip install rapidfuzz


In [96]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from rapidfuzz import process, fuzz

In [97]:

def clean_anime_name(name):
    if pd.isnull(name): return ""
    name = re.sub(r"Season\s*\d+", "", name, flags=re.IGNORECASE)
    name = re.sub(r"Final\s*Season.*", "", name, flags=re.IGNORECASE)
    name = re.sub(r"Part\s*\d+", "", name, flags=re.IGNORECASE)
    name = re.sub(r"[^a-zA-Z0-9\s]", "", name)
    return name.strip().lower()

def normalize(text): return text.strip().lower()

In [98]:
# TV ve Movie veri setlerine güvenli bir şekilde 'source' ekle
tv_final_df = tv_final_df.copy()
tv_final_df["source"] = "tv"

movie_final_df = movie_final_df.copy()
movie_final_df["source"] = "movie"
remove_types = ["Special", "OVA", "ONA", "Music", "Recap"]

def clean_df(df):
    return df[
        df["synopsis"].notnull() &
        df["name"].notnull() &
        (df["score"].fillna(0) >= 5.0) &
        (~df["type"].isin(remove_types)) &
        (df["episodes"].fillna(2) > 1)
    ]

tv_final_df = clean_df(tv_final_df)
movie_final_df = clean_df(movie_final_df)

# Birleştir ve yeni df_all'u garanti bir kopya olarak al
df_all = pd.concat([tv_final_df, movie_final_df], ignore_index=True).copy()

# Güvenli şekilde temizleme işlemleri
df_all["base_name"] = df_all["name"].apply(clean_anime_name).str.lower()

df_all["combined_text"] = (
    df_all["synopsis"].fillna("") + " " +
    df_all["genres"].fillna("") + " " +
    df_all["type"].fillna("") + " " +
    df_all["rating"].fillna("")
).str.lower()


In [99]:
from rapidfuzz import process, fuzz

def fuzzy_merge(df_base, df_additional, on='base_name', threshold=90):
    matches = []
    for name in df_base[on]:
        match = process.extractOne(name, df_additional['English name cleaned'], scorer=fuzz.token_sort_ratio)
        if match and match[1] >= threshold:
            row = df_additional[df_additional['English name cleaned'] == match[0]]
            if not row.empty:
                row_data = {
                    "Score": row["Score"].values[0] if "Score" in row else np.nan,
                    "Rank": row["Rank"].values[0] if "Rank" in row else np.nan,
                    "Popularity": row["Popularity"].values[0] if "Popularity" in row else np.nan
                }
                matches.append(pd.Series(row_data))
            else:
                matches.append(pd.Series({"Score": np.nan, "Rank": np.nan, "Popularity": np.nan}))
        else:
            matches.append(pd.Series({"Score": np.nan, "Rank": np.nan, "Popularity": np.nan}))
    scores_df = pd.DataFrame(matches).reset_index(drop=True)
    return pd.concat([df_base.reset_index(drop=True), scores_df], axis=1)


In [100]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix_all = vectorizer.fit_transform(df_all["combined_text"])
# anime-dataset-2023.csv dosyasını oku
df2 = pd.read_csv("anime-dataset-2023.csv")
df2["English name cleaned"] = df2["English name"].fillna(df2["Name"]).apply(clean_anime_name)
# Gereksiz kolonları sil
drop_cols = [
    "Producers", "Licensors", "Premiered",
    "Broadcast", "Opening Theme", "Ending Theme", "Background"
]
df_all.drop(columns=[col for col in drop_cols if col in df_all.columns], inplace=True)
# Merge et
df_all = fuzzy_merge(df_all, df2)
df_all["Score"] = pd.to_numeric(df_all["Score"], errors="coerce")

# Satır bazlı temizlik
df_all = df_all[df_all["synopsis"].notnull() & df_all["name"].notnull()]
df_all = df_all[df_all["Score"].fillna(0) > 4]






In [101]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Gerekli kolonlar
required_cols = ["Score", "Rank", "Popularity"]

# Eksik olan kolonları ekle ve tipleri dönüştür
for col in required_cols:
    if col not in df_all.columns:
        df_all[col] = np.nan

# Tip kontrolü: Sayısal değilse dönüştür
for col in required_cols:
    df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

# Ortalamayla doldur
df_all[required_cols] = df_all[required_cols].fillna(df_all[required_cols].mean())

# Normalize et
scaler = MinMaxScaler()
df_all[["norm_score", "norm_rank", "norm_popularity"]] = scaler.fit_transform(df_all[required_cols])

# Quality Score hesapla (score yüksek, rank/popularity düşük iyidir)
df_all["quality_score"] = (
    df_all["norm_score"] * 0.5 +
    (1 - df_all["norm_rank"]) * 0.3 +
    (1 - df_all["norm_popularity"]) * 0.2
)


In [159]:
from rapidfuzz import process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# 1️⃣ Kullanılacak veriyi filtrele (örneğin synopsis kısa olanları at)
df_filtered = df_all[df_all["synopsis"].str.len() >= 30].copy()
df_filtered = df_filtered.reset_index(drop=True)  # index karışıklığını önlemek için sıfırla

# 2️⃣ TF-IDF matrisini oluştur
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix_filtered = vectorizer.fit_transform(df_filtered["combined_text"])

# 3️⃣ Kullanıcıdan anime isimleri al
print("📥 Lütfen en az 1, en fazla 5 anime ismi giriniz.")
user_inputs = []
while len(user_inputs) < 5:
    anime_name = input(f"{len(user_inputs)+1}. anime ismi (bitirmek için boş bırak ve Enter'a bas): ").strip()
    if anime_name == '':
        break
    user_inputs.append(anime_name)

if not user_inputs:
    print("❌ En az bir anime ismi girmelisiniz.")
    exit()

# 4️⃣ Anime isimlerini fuzzy match ile eşleştir
matched_indices = []
for anime in user_inputs:
    cleaned_input = normalize(anime)
    best_match = process.extractOne(cleaned_input, df_filtered["base_name"].tolist(), score_cutoff=80)
    if best_match:
        idx = df_filtered[df_filtered["base_name"] == best_match[0]].index
        if not idx.empty:
            matched_indices.append(idx[0])
        print(f"✅ '{anime}' eşleşti → '{best_match[0]}'")
    else:
        print(f"⚠️ '{anime}' için eşleşme bulunamadı.")

if not matched_indices:
    print("❌ Hiçbir geçerli eşleşme bulunamadı.")
    exit()

# 5️⃣ TF-IDF ortalama vektörünü hesapla
input_vecs = tfidf_matrix_filtered[matched_indices]
mean_vec = input_vecs.mean(axis=0)
mean_vec = np.array(mean_vec)

# 6️⃣ Cosine similarity ile benzerlikleri bul
cosine_sim = cosine_similarity(mean_vec, tfidf_matrix_filtered).flatten()

# 7️⃣ Final skoru hesapla (benzerlik ve kalite skoru birlikte)
scored = []
for i, sim in enumerate(cosine_sim):
    qual = df_filtered.iloc[i]['quality_score'] if not pd.isna(df_filtered.iloc[i]['quality_score']) else 0
    final_score = 0.8 * sim + 0.2 * qual
    scored.append((i, final_score))


# 8️⃣ En iyi 10 sonucu al (kullanıcının girdiklerini çıkar)
top10 = sorted(scored, key=lambda x: x[1], reverse=True)
top10 = [item for item in top10 if item[0] not in matched_indices][:10]

pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_columns", None)
# 9️⃣ Önerileri yazdır
print("\n🎯 Final Skor ile En İyi 10 Anime Önerisi:")
recommendations = df_filtered.iloc[[i for i, _ in top10]][['name', 'score', 'genres', 'quality_score']]
recommendations.reset_index(drop=True, inplace=True)
print(recommendations)


📥 Lütfen en az 1, en fazla 5 anime ismi giriniz.


1. anime ismi (bitirmek için boş bırak ve Enter'a bas):  91 days
2. anime ismi (bitirmek için boş bırak ve Enter'a bas):   


✅ '91 days' eşleşti → '91 days'

🎯 Final Skor ile En İyi 10 Anime Önerisi:
                                    name  score                                             genres  quality_score
0  JoJo's Bizarre Adventure: Golden Wind   8.58                                  Action, Adventure       0.932779
1                                 Erased   8.30                                  Mystery, Suspense       0.897636
2        The Eminence in Shadow Season 2   8.29                            Action, Comedy, Fantasy       0.895557
3                      The Tatami Galaxy   8.55  Award Winning, Comedy, Mystery, Romance, Suspense       0.927375
4        Fruits Basket: The Final Season   8.95                       Drama, Romance, Supernatural       0.981753
5                            Banana Fish   8.45                 Action, Adventure, Drama, Suspense       0.918834
6                 The Promised Neverland   8.48                                  Mystery, Suspense       0.924767
7            

In [104]:
print(df_all[['Score', 'Rank', 'Popularity']].head())


   Score   Rank  Popularity
1   9.10    1.0         3.0
2   9.07    3.0        13.0
3   8.94   16.0       138.0
4   8.54  107.0         1.0
5   8.41  170.0       357.0


In [105]:
df[df['synopsis'].str.len() < 30].shape

(8233, 21)

In [153]:
print("TF-IDF shape:", tfidf_matrix_all.shape)
print("df_all shape:", df_all.shape)


TF-IDF shape: (4392, 29401)
df_all shape: (2723, 31)


In [161]:
df_filtered.to_csv("anime_cleaned.csv", index=False)


In [165]:
import streamlit as st
import pandas as pd
import numpy as np
from rapidfuzz import process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load your preprocessed anime dataset here
@st.cache_data
def load_data():
    df = pd.read_csv("anime_cleaned.csv")  # senin temizlenmiş df_filtered.csv
    df = df[df["synopsis"].str.len() >= 30].copy()
    df = df.reset_index(drop=True)
    return df

df = load_data()

# TF-IDF vectorizer
@st.cache_resource
def vectorize_text(df):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df['combined_text'])
    return vectorizer, tfidf_matrix

vectorizer, tfidf_matrix = vectorize_text(df)

# Normalizasyon
def normalize(text):
    return ''.join(e.lower() for e in text if e.isalnum() or e.isspace()).strip()

# Başlık
st.title("🎌 Anime Öneri Sistemi")
st.write("Beğendiğiniz 1-5 animeyi girin, size benzeyenleri önerelim.")

# Giriş alanı
anime_inputs = []
for i in range(5):
    name = st.text_input(f"{i+1}. Anime Adı", "")
    if name.strip():
        anime_inputs.append(name.strip())

# Buton
if st.button("🎯 Önerileri Getir"):
    if not anime_inputs:
        st.warning("Lütfen en az 1 anime ismi girin.")
    else:
        matched_indices = []
        for anime in anime_inputs:
            cleaned_input = normalize(anime)
            match = process.extractOne(cleaned_input, df["base_name"].tolist(), score_cutoff=80)
            if match:
                idx = df[df["base_name"] == match[0]].index
                if not idx.empty:
                    matched_indices.append(idx[0])
                    st.success(f"✅ '{anime}' eşleşti → '{match[0]}'")
            else:
                st.warning(f"⚠️ '{anime}' için eşleşme bulunamadı.")

        if not matched_indices:
            st.error("Hiçbir eşleşme bulunamadı.")
        else:
            # TF-IDF vektörü
            input_vecs = tfidf_matrix[matched_indices]
            mean_vec = input_vecs.mean(axis=0)
            mean_vec = np.array(mean_vec)

            # Benzerlik
            cosine_sim = cosine_similarity(mean_vec, tfidf_matrix).flatten()

            # Skor hesapla
            scored = []
            for i, sim in enumerate(cosine_sim):
                if i not in matched_indices:
                    qual = df.iloc[i]['quality_score'] if not pd.isna(df.iloc[i]['quality_score']) else 0
                    score = 0.8 * sim + 0.2 * qual
                    scored.append((i, score))

            # En iyi 10 öneri
            top10 = sorted(scored, key=lambda x: x[1], reverse=True)[:10]
            results = df.iloc[[i for i, _ in top10]][['name', 'score', 'genres', 'quality_score']].reset_index(drop=True)

            st.subheader("🔝 En İyi 10 Öneri:")
            st.dataframe(results)


2025-07-21 14:54:08.995 No runtime found, using MemoryCacheStorageManager
